# <h1><centre> RFM Analysis using Customer Segmentation </centre></h1>

## Business Problem

* An e-commerce company aims to separate its customers into different segments and implement a market strategy according to those segments.

* In order to achieve this goal, purchasing actions of different customers will be clarified and customers will be grouped according to those actions

## Recency, Frequency and Monetary Analysis(RFM)

RFM is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

  RFM stands for the three dimensions:

    * Recency – How recently did the customer purchase?

    * Frequency – How often do they purchase?

    * Monetary – How much do they spend?

### Importing Llibraries

In [1]:
# Data manipulation and linear algebra
import pandas as pd
# Date
import datetime as dt
# Settings
import warnings

In [5]:
df = pd.read_excel('C:/Users/MILAN/Downloads/online_retail_II.xlsx')

In [6]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


### Data Preprocessing

In [8]:
# Removing returned products (Invoice numbers starting with C) from the data set
df = df[~df["Invoice"].str.contains("C", na = False)]
# Removing missing values from the dataset
df.dropna(inplace = True)

## RFM Analysis

### Recency

In order to find the recency value of each customer, we need to determine the last invoice date as the current date and subtract the last purchasing date of each customer from this date.

In [9]:
df["InvoiceDate"].max() # Last invoice date

Timestamp('2010-12-09 20:01:00')

In [10]:
today_date = dt.datetime(2011,12,9) # last invoice date is assigned to today_date variable

In [11]:
# The type of Customer ID variable needs to be turned into an integer for following commands.
df["Customer ID"] = df["Customer ID"].astype(int) 

In [12]:
# The type of InvoiceDate variable needs to be turned into datetime for following commands.
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [13]:
# Grouping the last invoice dates according to the Customer ID variable, subtracting them from today_date, and assigning them as recency
recency = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))
# Rename column name as Recency
recency.rename(columns = {"InvoiceDate":"Recency"}, inplace = True)
# Change the values to day format
recency_df = recency["Recency"].apply(lambda x: x.days)
recency_df.head()

Customer ID
12346    528
12347    366
12348    437
12349    406
12351    374
Name: Recency, dtype: int64

### Frequency

In order to find the frequency value of each customer, we need to determine how many times the customers make purchases.

In [14]:
# Grouping unique values of invoice date according to customer_id variable and assigning them to freq_df variable
freq_df = df.groupby("Customer ID").agg({"InvoiceDate":"nunique"}) 
# Rename column name as Frequency
freq_df.rename(columns={"InvoiceDate": "Frequency"}, inplace=True)
freq_df.head()

,Frequency
Customer ID,
12346,11
12347,2
12348,1
12349,3
12351,1


### Monetary

In order to find the monetary value of each customer, we need to determine how much do the customers spend on purchases

In [15]:
# Multiplying the prices and quantities of purchased products and assigning them to the total price variable
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [16]:
# Grouping and sum up total prices according to each Customer ID
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
# Rename Total Price column as Monetary
monetary_df.rename(columns={"TotalPrice":"Monetary"}, inplace=True)
monetary_df.head()

,Monetary
Customer ID,
12346,372.86
12347,1323.32
12348,222.16
12349,2671.14
12351,300.93


### Concatenate Recency,Frequency and Monetary

In [17]:
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,528,11,372.86
12347,366,2,1323.32
12348,437,1,222.16
12349,406,3,2671.14
12351,374,1,300.93


### Scoring of Recency, Frequency and Monetary Values

In [18]:
# Dividing the recency values into recency scores such that the lowest recency value as 5 and the highest as 1
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4 , 3, 2, 1]) 
# Dividing the frequency values into frequency scores such that the lowest frequency value as 1 and the highest as 5
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
# Dividing the monetary values into monetary scores such that the lowest monetary value as 1 and the highest as 5
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

In [19]:
# Combining Recency, Frequency, and Monetary Scores in a string format
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) + 
                    rfm['FrequencyScore'].astype(str) + 
                    rfm['MonetaryScore'].astype(str))

In [20]:
# Customers with best scores
rfm[rfm["RFM_SCORE"]=="555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12415,374,7,19543.84,5,5,5,555
12431,372,13,4370.52,5,5,5,555
12471,373,48,20139.74,5,5,5,555
12472,368,13,11308.48,5,5,5,555
12474,377,13,5048.66,5,5,5,555


In [21]:
# Customers with worst scores
rfm[rfm["RFM_SCORE"]=="111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12362,737,1,130.00,1,1,1,111
12392,590,1,234.75,1,1,1,111
12404,681,1,63.24,1,1,1,111
12416,655,1,202.56,1,1,1,111
12466,681,1,56.73,1,1,1,111


## Customer Segmentation

In [22]:
# Mapping of segments according to recency and frequency scores of customers
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [23]:
# Recency and Frequency scores are turned into string format, combined and assigned to Segment
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
# Segments are changed with the definitons of seg_map
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

In [24]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,528,11,372.86,2,5,2,252,Can't Loose
12347,366,2,1323.32,5,2,4,524,Potential Loyalists
12348,437,1,222.16,2,1,1,211,Hibernating
12349,406,3,2671.14,3,3,5,335,Need Attention
12351,374,1,300.93,5,1,2,512,New Customers


In [25]:
# Mean, median, count statistics of different segments
rfm[["Segment","Recency","Frequency", "Monetary"]].groupby("Segment").agg(["mean","median","count"])

Recency               Frequency               \
                           mean median count       mean median count   
Segment                                                                
About to Sleep       416.851312  416.0   343   1.201166    1.0   343   
At Risk              514.937807  493.0   611   3.070376    3.0   611   
Can't Loose          486.717949  471.5    78   9.038462    7.5    78   
Champions            370.119155  370.0   663  12.499246    8.0   663   
Hibernating          577.309055  576.0  1016   1.126969    1.0  1016   
Loyal Customers      399.275908  394.0   743   6.816958    5.0   743   
Need Attention       416.212560  416.0   207   2.454106    2.0   207   
New Customers        371.580000  371.5    50   1.000000    1.0    50   
Potential Loyalists  381.773256  382.0   516   2.023256    2.0   516   
Promising            388.758621  388.0    87   1.000000    1.0    87   

                        Monetary                  
                            mean    median count  
Segment                                           
About to Sleep        442.266851   317.760   343  
At Risk              1188.207056   760.190   611  
Can't Loose          4072.967051  2316.485    78  
Champions            6852.264167  2508.320   663  
Hibernating           401.982150   250.155  1016  
Loyal Customers      2743.484342  1818.710   743  
Need Attention       1058.788068   730.470   207  
New Customers         386.199200   258.825    50  
Potential Loyalists   729.164806   523.660   516  
Promising             368.020690   293.740    87

Several marketing strategies can be determined for different customer segments. I have determined 3 strategies for different customer segments. These can be diversified and customers can be monitored more closely.

### At Risk

Those in this group last shopping an average of 371 days ago. The group median was 375.0, so there was not much deviation from the mean. Therefore, it can be said that this number is consistent throughout the group. On average, 3.89 units of shopping were made and 1379.64 units of payments were made. The time interval that has passed since the last purchase of this group is very high, so customers may be lost. The reasons that may cause these people not to shop for so long should be focused on. There may be a case of customer dissatisfaction. The shopping experience of the customer can be examined by sending a survey via mail. If there is no dissatisfaction, then the person is reminded. Options such as discount codes may be offered to encourage re-shopping.

### Need Attention

People in this group last shopping, on average, 112 days ago. The group median is 105, so there is not much deviation from the mean. Hence, this number is consistent across the group. On average, 3.14 units of shopping were made and 1276.34 units of payment were made. This group is less risky than the At-Risk group. The last shopping date is relatively close. Special offers can be made from products whose consumption is faster than among the products that those customers shop. By doing this, the average visit time of customers can be shortened.

### Potential Loyalists

Those in this group last shopping an average of 24 days ago. The group median is 22, so there is not much deviation from the mean. Hence, this number is consistent across the group. On average, 2.58 units were purchased and 1158.27 units were paid. People in this group can be included in the Loyal Customer group if supported. Therefore, they can be monitored closely and customer satisfaction can be increased with one-to-one phone calls. Apart from this, options such as free shipping can be offered to increase the average paid wages.